In [20]:
!python --version

Python 3.12.3


In [ ]:
os.path.join('.', 'daily_push', 'data', 'Gangnam3Districts_Boundary_Center.pickle')

In [19]:
import pickle
import os
with open(os.path.join('.', 'daily_push', 'data', 'Gangnam3Districts_Boundary_Center.pickle'), 'rb') as f:
    강남3구_집계구_경계_중심 = pickle.load(f)
강남3구_집계구_경계_중심

/tmp/ipykernel_3700397/3247070780.py:4: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  강남3구_집계구_경계_중심 = pickle.load(f)


,gu,TOT_REG_CD,geometry,count,centroid
0,11220,11220520010107,"POLYGON ((127.02654 37.48948, 127.02601 37.489...",53.0,POINT (127.02621 37.48978)
1,11220,11220590020102,"POLYGON ((127.00201 37.5079, 127.00223 37.5074...",54.0,POINT (127.00173 37.50763)
2,11220,11220670020002,"POLYGON ((127.03551 37.46515, 127.03539 37.464...",114.0,POINT (127.0478 37.44425)
3,11220,11220680030001,"POLYGON ((127.07618 37.45624, 127.07639 37.456...",116.0,POINT (127.06819 37.44481)
4,11220,11220540010106,"POLYGON ((127.02177 37.50081, 127.02166 37.500...",60.0,POINT (127.02117 37.50097)
...,...,...,...,...,...
3067,11240,11240540010801,"POLYGON ((127.14493 37.48965, 127.14463 37.488...",146.0,POINT (127.14401 37.48973)
3068,11240,11240590030105,"POLYGON ((127.14341 37.51721, 127.14356 37.517...",63.0,POINT (127.14088 37.51645)
3069,11240,11240590040102,"POLYGON ((127.13568 37.51516, 127.13544 37.514...",51.0,POINT (127.13466 37.51537)
3070,11240,11240820010202,"POLYGON ((127.14042 37.48346, 127.14018 37.482...",66.0,POINT (127.13148 37.47358)


In [46]:
import requests
공휴일_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
공휴일_params ={'serviceKey' : 'z1DJA6Ag86kdealhvEpwafaXLI8NEGWSThqDqdsBzHaeGYVI83kcGrdTnAfRBcohutSUt2l6/NSYR6lMF31ZNg==', 
             # z1DJA6Ag86kdealhvEpwafaXLI8NEGWSThqDqdsBzHaeGYVI83kcGrdTnAfRBcohutSUt2l6%2FNSYR6lMF31ZNg%3D%3D
         'solYear' : '2022', 'numOfRows' : '30', 
             # '_type': 'json' 
            }
공휴일_response = requests.get(공휴일_url, params = 공휴일_params)
result = 공휴일_response.content
result


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><dateKind>01</dateKind><dateName>1\xec\x9b\x941\xec\x9d\xbc</dateName><isHoliday>Y</isHoliday><locdate>20220101</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>\xec\x84\xa4\xeb\x82\xa0</dateName><isHoliday>Y</isHoliday><locdate>20220131</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>\xec\x84\xa4\xeb\x82\xa0</dateName><isHoliday>Y</isHoliday><locdate>20220201</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>\xec\x84\xa4\xeb\x82\xa0</dateName><isHoliday>Y</isHoliday><locdate>20220202</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>\xec\x82\xbc\xec\x9d\xbc\xec\xa0\x88</dateName><isHoliday>Y</isHoliday><locdate>20220301</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>\xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9\xec\x84\xa0\xea\xb1\xb0\xec\

In [44]:
import os
import sys
working_dir = os.getcwd()

sys.path.append(working_dir)
from model_train_deploy.lib.db_io import Engine
from model_train_deploy.lib.utils import 구분데이터붙이기, 군집나누기
from model_train_deploy.lib.simulator_02 import Cluster
import pandas as pd
import pickle
from scipy.stats import ks_2samp
from sqlalchemy import text

db_info = {'host': '163.152.172.163',
          'port': '5432',
          'db_name': 'postgres',
          'user_name': 'tgsociety',
          'password': 'tgsociety'}
execution_date = '2024-07-19 23:00:00+09'




# def measure_drift(execution_date):
execution_date = pd.to_datetime(execution_date)
db_handler = Engine(db_info)

query = '''
    SELECT *
    FROM datamart.history_학습id
    ORDER BY 등록일시 DESC
    LIMIT 1
'''
train_history = pd.read_sql(query, db_handler.engine)

new_query = '''
SELECT *
FROM datamart.고객수요
WHERE 예약희망일시 >= %(st_dt)s
    and 예약희망일시 <= %(ed_dt)s
'''
# 새로운 데이터 불러오기
ref_data = pd.read_sql(new_query, db_handler.engine, params={'st_dt': train_history.loc[0, 'train_data_start_dt'], 'ed_dt': train_history.loc[0, 'train_data_end_dt']})
ref_data['예약희망일시'] = ref_data['예약희망일시'].dt.tz_convert('Asia/Seoul')
ref_data['고객_접속일시'] = ref_data['고객_접속일시'].dt.tz_convert('Asia/Seoul')

cur_data = pd.read_sql(new_query, db_handler.engine, params={'st_dt': train_history.loc[0, 'train_data_start_dt'], 'ed_dt': execution_date})
cur_data['예약희망일시'] = cur_data['예약희망일시'].dt.tz_convert('Asia/Seoul')
cur_data['고객_접속일시'] = cur_data['고객_접속일시'].dt.tz_convert('Asia/Seoul')
query = '''
SELECT demand_cluster_pickle
FROM datamart.storage_demand_cluster
WHERE train_id = %(train_id)s
'''

# 쿼리 실행 및 데이터 불러오기
tmp_df = pd.read_sql(query, db_handler.engine, params={'train_id': int(train_history.loc[0, 'train_id'])})

클러스터링결과 = pickle.loads(tmp_df.loc[0, 'demand_cluster_pickle'])
# 클러스터링결과['cluster'] = self.cluster
# 클러스터링결과['hierarchy_sample_data'] = self.hierarchy_sample_data

ref_data = 구분데이터붙이기(ref_data.copy(), '예약희망일시')
cur_data = 구분데이터붙이기(cur_data.copy(), '예약희망일시')

군집_ref_data = 군집나누기(ref_data.copy(), 클러스터링결과['cluster'].군집s)
군집_cur_data = 군집나누기(cur_data.copy(), 클러스터링결과['cluster'].군집s)

p_values = []
for key in list(군집_ref_data):
    if (len(군집_ref_data[key])==0) & (len(군집_cur_data[key])==0):
        p_values.append(1)
    elif (len(군집_ref_data[key])==0) | (len(군집_cur_data[key])==0):
        p_values.append(0)
    else:
        p_values.append(ks_2samp(군집_ref_data[key][:int(len(군집_ref_data[key])/2)], 군집_cur_data[key])[1].item())

        
data_start_dt = train_history.loc[0, 'train_data_start_dt']
data_end_dt = cur_data.sort_values(by = '예약희망일시', ascending = False).reset_index(drop=True).loc[0, '예약희망일시']

insert_query = text('''INSERT INTO datamart."history_드리프트id" ("drift_data_start_dt", "drift_data_end_dt", "logical_dt") VALUES (:st, :ed, :logi) RETURNING "drift_id";''')

with db_handler.engine.connect() as connection:
    trans = connection.begin()
    result = connection.execute(insert_query, {'st': data_start_dt, 'ed': data_end_dt, 'logi': execution_date})
    
    # 생성된 id 값을 가져옴
    drift_id = result.scalar()
    trans.commit()

tmp_df = pd.DataFrame(columns = ['drift_id', 'train_id', 'cluster_num', 'drift_method', 'drift_value', 'drift_yn'])
cluster_nums = []
drift_values = []
drift_yns = []
for i, p in enumerate(p_values):
    cluster_nums.append(i)
    drift_values.append(p)
    drift_yns.append(p <= 0.05)
tmp_df = pd.DataFrame({
    'drift_id': [drift_id]*len(p_values),
    'train_id': [int(train_history.loc[0, 'train_id'])]*len(p_values),
    'cluster_num': cluster_nums,
    'drift_method': ['Kolmogorov_Smirnov Test']*len(p_values),
    'drift_value': drift_values,
    'drift_yn': drift_yns
})
tmp_df.to_sql('result_drift', db_handler.engine, schema = 'datamart', index = False, if_exists='append')

/home/yspark/driving-teacher-ai/dags/model_train_deploy/lib/utils.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hierarchy_sample_data[key] = pd.concat([hierarchy_sample_data[key],
/home/yspark/driving-teacher-ai/dags/model_train_deploy/lib/utils.py:231: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hierarchy_sample_data[key] = pd.concat([hierarchy_sample_data[key],
/home/yspark/driving-teacher-ai/dags/model_train_deploy/lib/utils.py:231: FutureWarning: The behavior of DataFrame concatena

17

In [15]:
from scipy.stats import ks_2samp
ks_2samp([0,0,1,0,0]*30, [0,0,0,0,1,1]*10)[1].item()

0.40355354646300684

In [41]:
drift_id

1

In [18]:

for key in list(군집_ref_data):
    # if len(군집_ref_data[key]) >1:
    print(ks_2samp(군집_ref_data[key][:int(len(군집_ref_data[key])/2)], 군집_cur_data[key])[1])

ValueError: Data passed to ks_2samp must not be empty

In [ ]:
def 군집나누기(data, 군집):
    '''
    data는 구분데이터붙이기가 돼어있어야함.
    군집은 cluster.군집s를 불러와야함.
    '''
    hierarchy_sample_data = {}
    for key in list(군집):
        hierarchy_sample_data[key] = pd.DataFrame(columns = ['고객_접속일시', '예약희망일시', '예약희망_point', '연월일', '연', '월', '요일', '공휴일', '시간대'])
        for condition in eval(key):
            condition1 = (data['월'] == condition[0])
            condition2 = (data['요일'] == condition[1])
            condition3 = (data['공휴일'] == condition[2])
            condition4 = (data['시간대'] == condition[3])
            hierarchy_sample_data[key] = pd.concat([hierarchy_sample_data[key], 
                                                    data.loc[condition1&condition2&condition3&condition4, :]]).reset_index(drop = True)
    return hierarchy_sample_data

In [13]:
df

,train_id,train_data_start_dt,train_data_end_dt,등록일시
0,48,2022-06-28 01:00:00+00:00,2024-07-19 08:54:31.089964+00:00,2024-07-19 13:39:14.850626+00:00


In [14]:
ref_data

,예약희망일시,고객_접속일시,고객_접속point,예약희망_point,고객_코드
0,2022-07-01 09:00:00+00:00,2022-06-19 22:28:21.496006+00:00,0101000020E61000006125759E75C85F40532CCFAB3DBD...,0101000020E61000006125759E75C85F40532CCFAB3DBD...,70
1,2022-07-01 07:00:00+00:00,2022-06-25 05:01:23.216783+00:00,0101000020E61000002FB4A4CBD0C25F405F44093665BF...,0101000020E61000002FB4A4CBD0C25F405F44093665BF...,53
2,2022-07-01 08:00:00+00:00,2022-06-25 08:36:34.195153+00:00,0101000020E6100000E652EDFFFFBE5F40D7D7AA753DBE...,0101000020E6100000E652EDFFFFBE5F40D7D7AA753DBE...,64
3,2022-07-01 00:30:00+00:00,2022-06-27 08:37:27.256670+00:00,0101000020E61000004FEE9DBDFDC65F40E07A15FFD7C2...,0101000020E61000004FEE9DBDFDC65F40E07A15FFD7C2...,2
4,2022-07-01 00:30:00+00:00,2022-06-27 13:31:05.356583+00:00,0101000020E6100000645886C5D7C75F406F54223C8BC0...,0101000020E6100000645886C5D7C75F406F54223C8BC0...,0
...,...,...,...,...,...
62003,2024-07-18 11:00:00+00:00,2024-07-17 09:23:23.784285+00:00,0101000020E610000031DE189B88C15F40F59C6F2776C0...,0101000020E610000031DE189B88C15F40F59C6F2776C0...,72
62004,2024-07-18 11:30:00+00:00,2024-07-17 09:47:31.481653+00:00,0101000020E61000003C4882FCABC95F40F6BEC5E7B6BF...,0101000020E61000003C4882FCABC95F40F6BEC5E7B6BF...,76
62005,2024-07-18 08:30:00+00:00,2024-07-17 09:48:37.582923+00:00,0101000020E61000007C1B88E1EAC05F4005D1073707BB...,0101000020E61000007C1B88E1EAC05F4005D1073707BB...,43
62006,2024-07-18 12:00:00+00:00,2024-07-17 10:12:34.110915+00:00,0101000020E61000008F819CD063C75F40CB1557E39FBF...,0101000020E61000008F819CD063C75F40CB1557E39FBF...,80


In [15]:
cur_data

,예약희망일시,고객_접속일시,고객_접속point,예약희망_point,고객_코드
0,2022-07-01 09:00:00+00:00,2022-06-19 22:28:21.496006+00:00,0101000020E61000006125759E75C85F40532CCFAB3DBD...,0101000020E61000006125759E75C85F40532CCFAB3DBD...,70
1,2022-07-01 07:00:00+00:00,2022-06-25 05:01:23.216783+00:00,0101000020E61000002FB4A4CBD0C25F405F44093665BF...,0101000020E61000002FB4A4CBD0C25F405F44093665BF...,53
2,2022-07-01 08:00:00+00:00,2022-06-25 08:36:34.195153+00:00,0101000020E6100000E652EDFFFFBE5F40D7D7AA753DBE...,0101000020E6100000E652EDFFFFBE5F40D7D7AA753DBE...,64
3,2022-07-01 00:30:00+00:00,2022-06-27 08:37:27.256670+00:00,0101000020E61000004FEE9DBDFDC65F40E07A15FFD7C2...,0101000020E61000004FEE9DBDFDC65F40E07A15FFD7C2...,2
4,2022-07-01 00:30:00+00:00,2022-06-27 13:31:05.356583+00:00,0101000020E6100000645886C5D7C75F406F54223C8BC0...,0101000020E6100000645886C5D7C75F406F54223C8BC0...,0
...,...,...,...,...,...
62003,2024-07-18 11:00:00+00:00,2024-07-17 09:23:23.784285+00:00,0101000020E610000031DE189B88C15F40F59C6F2776C0...,0101000020E610000031DE189B88C15F40F59C6F2776C0...,72
62004,2024-07-18 11:30:00+00:00,2024-07-17 09:47:31.481653+00:00,0101000020E61000003C4882FCABC95F40F6BEC5E7B6BF...,0101000020E61000003C4882FCABC95F40F6BEC5E7B6BF...,76
62005,2024-07-18 08:30:00+00:00,2024-07-17 09:48:37.582923+00:00,0101000020E61000007C1B88E1EAC05F4005D1073707BB...,0101000020E61000007C1B88E1EAC05F4005D1073707BB...,43
62006,2024-07-18 12:00:00+00:00,2024-07-17 10:12:34.110915+00:00,0101000020E61000008F819CD063C75F40CB1557E39FBF...,0101000020E61000008F819CD063C75F40CB1557E39FBF...,80


In [ ]:
train_id, cluster_num, drift_method, drift_value, drift_yn, logical_dt, 적재일시